In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib
from matplotlib import pyplot as plt
from textwrap import wrap


pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'

import pickle
import time
from tqdm import tqdm

from itertools import product

#import googletrans
#from googletrans import Translator
#to avoid instantiate too many translator, we created one instance for the whole notebook
#translator = Translator()


from prophet import Prophet
import itertools
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import holidays


import hts.functions
import collections
from hts import HTSRegressor
from hts.hierarchy import HierarchyTree


from sktime.transformations.hierarchical.reconcile import Reconciler
from sktime.transformations.hierarchical.aggregate import Aggregator
from sktime.forecasting.exp_smoothing import ExponentialSmoothing


import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

# define functions

In [11]:
def get_best_hyperparams(data):   
    
    default_best_params = {'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.5}
    
    first_cutoff_date = pd.to_datetime('2014-02-1')
    first_nonzero_date = data['y'].ne(0).idxmax()
    
    # return default_best_params if 
    # 1. all values are zero
    # 2. too many values are zero
    if (data['y'] == 0).all() or (first_cutoff_date < first_nonzero_date):
        print( first_nonzero_date)
        print("too many zero values, return default best params")
        return default_best_params
    else:
        param_grid = {  
                'changepoint_prior_scale': [0.01,0.2],
                'seasonality_prior_scale': [0.01,0.5 ],
        }

        # Generate all combinations of parameters
        all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
        rmses = []  # Store the RMSEs for each params here

        # Use cross validation to evaluate all parameters
        cutoffs = pd.to_datetime(['2014-2-1', '2015-6-1'])

        for params in all_params:
            m = Prophet(**params).fit(data)  # Fit model with given params
            df_cv = cross_validation(m, cutoffs=cutoffs, horizon='90 days', parallel="processes")
            df_p = performance_metrics(df_cv, rolling_window=1)
            rmses.append(df_p['rmse'].values[0])

        # Find the best parameters
        tuning_results = pd.DataFrame(all_params)
        tuning_results['rmse'] = rmses

        best_params = all_params[np.argmin(rmses)]
        print(best_params)
        return best_params


In [12]:
def get_best_preds(df_,   #df_heir_train
                   cat_id  #cat_id in oneshop_cat_list 
                  ):
    #get dataframe
    columns = ['ds', cat_id]
    df = df_.loc[:, columns]
    df.columns = ['ds', 'y']
    
    #get best_params
    best_params = get_best_hyperparams(df)
    
    #create model based on best_params
    m = Prophet(changepoint_prior_scale=best_params['changepoint_prior_scale'], 
                seasonality_prior_scale=best_params['seasonality_prior_scale'], 
                yearly_seasonality=True)
    m.add_country_holidays(country_name='RU')
    m.fit(df)

    #predict and return
    future = m.make_future_dataframe(periods=1, freq='MS')
    forecast = m.predict(future)

    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(3))
    #fig1 = m.plot(forecast)

    best_preds = forecast.tail(1).yhat.values[0]
    print('best_preds: ', best_preds)
    return best_preds
    

# import data from pickle file

In [2]:
pickle_dict2 = pickle.load(open('../temp/data2.pkl', 'rb'))
df_basegrid = pickle_dict2['df_basegrid']   #concat df_train_m and df_test


In [5]:
list_shop_id=sorted(df_basegrid['shop_id'].unique())
print(list_shop_id)
print(len(list_shop_id))

[2, 3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 26, 28, 31, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59]
42


### shop_id: 36 only have one month data, won't be feed into forcasting

In [8]:
list_shop_id_filtered = list_shop_id.copy()
list_shop_id_filtered.remove(36)
print(list_shop_id_filtered)
print(len(list_shop_id_filtered))

[2, 3, 4, 5, 6, 7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 26, 28, 31, 34, 35, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59]
41


### forecasting all category sales in all shops

In [17]:
%%time
    
for shop_id in tqdm(list_shop_id_filtered):
    
     print(f'\n\nforecasting shop: s{shop_id}')
    
    ####read pickle_dict, which store everything
    #############################################
    pickle_dict_oneshop = pickle.load(open(f'../temp/s{shop_id}_hier.pkl', 'rb'))
    df_hier  = pickle_dict_oneshop['df_hier']
    tree = pickle_dict_oneshop['tree']
    sum_mat = pickle_dict_oneshop['sum_mat']
    sum_mat_labels =pickle_dict_oneshop['sum_mat_labels']
    oneshop_cat_list = pickle_dict_oneshop['oneshop_cat_list']
    oneshop_item_list = pickle_dict_oneshop['oneshop_item_list']


    ################### creat train, validation,and test dataset
    ###########################################################################
    df_hier_train = df_hier.loc[df_hier.index <= '2015-10-01']
    df_hier_test = df_hier.loc[df_hier.index == '2015-11-01']


    forecasts = pd.DataFrame(columns = df_hier.columns, index=['fake'])
    ################### forecasting total sales in this shop
    #################################################################
    print('\ngetting best preds for forecast_total_sales')
    forecasts['total'] = [get_best_preds(df_hier_train, 'total')]

    
  
    ################### forecasting all category sales in this shop
    #################################################################
    #dataframe to hold the forecasts
    for cat in oneshop_cat_list:
        print('\ngetting best preds for ', cat)
        forecasts[cat] = [get_best_preds(df_hier_train, cat)]


    ###########save neccessary object to pickle_dict, 
    ############################################################################
    pickle_dict_oneshop['forecasts']=forecasts
    pickle.dump(pickle_dict_oneshop, open(f'../temp/s{shop_id}_forecasts.pkl', 'wb'))



  0%|                                                                                           | 0/42 [00:00<?, ?it/s]


getting best preds for  c0
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for  c11
2014-12-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.14       -0.23        0.48
33 2015-10-01  0.13       -0.23        0.50
34 2015-11-01  0.20       -0.16        0.57
best_preds:  0.19707196267481153

getting best preds for  c12
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  2.90       -9.56       15.54
33 2015-10-01  1.18      -10.98       13.76
34 2015-11-01 16.53        4.84       29.70
best_preds:  16.533476762249386

getting best preds for  c15
2014-06-01 00:00:00
too many zero values, return default best params
  

           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  2.25        0.66        3.85
33 2015-10-01  3.57        1.90        5.26
34 2015-11-01  4.16        2.37        5.80
best_preds:  4.161186896000797

getting best preds for  c42
2015-05-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  8.65        7.75        9.46
33 2015-10-01  8.72        7.88        9.63
34 2015-11-01 10.82        9.95       11.75
best_preds:  10.816719657063837

getting best preds for  c43
{'changepoint_prior_scale': 0.2, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.03       -0.48        0.55
33 2015-10-01  0.09       -0.45        0.61
34 2015-11-01  0.05       -0.49        0.53
best_preds:  0.04605302410733528

getting best preds for  c44
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.

{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  6.39        3.05        9.62
33 2015-10-01  6.15        2.83        9.52
34 2015-11-01  6.34        3.23        9.28
best_preds:  6.343388713166685

getting best preds for  c76
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for  c77
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 -0.02       -0.22        0.18
33 2015-10-01 -0.02       -0.24        0.19
34 2015-11-01 -0.02       -0.23        0.19
best_preds:  -0.023877221104369097

getting best preds for  c78
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lowe

  2%|█▉                                                                              | 1/42 [14:03<9:36:19, 843.40s/it]

...

           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 11.51        7.84       15.24
33 2015-10-01 13.86       10.37       17.72
34 2015-11-01 11.49        7.72       15.15
best_preds:  11.489225157341528

getting best preds for  c42
2015-05-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 11.22        9.28       13.10
33 2015-10-01  9.96        8.19       11.76
34 2015-11-01 10.60        8.58       12.51
best_preds:  10.604905325047131

getting best preds for  c43
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 -0.50       -3.35        2.42
33 2015-10-01 -0.72       -3.65        2.27
34 2015-11-01 -0.89       -3.81        1.99
best_preds:  -0.8940843648996212

getting best preds for  c44
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        

{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.5}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  4.71        2.61        6.91
33 2015-10-01  3.41        1.16        5.48
34 2015-11-01  7.12        5.07        9.18
best_preds:  7.1190271817215525

getting best preds for  c76
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01  0.00       -0.00        0.00
33 2015-10-01  0.00       -0.00        0.00
34 2015-11-01  0.00       -0.00        0.00
best_preds:  0.0

getting best preds for  c77
{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01}
           ds  yhat  yhat_lower  yhat_upper
32 2015-09-01 -0.06       -0.97        0.81
33 2015-10-01 -0.06       -0.93        0.81
34 2015-11-01 -0.05       -0.93        0.81
best_preds:  -0.05222589776565332

getting best preds for  c78
2013-01-01 00:00:00
too many zero values, return default best params
           ds  yhat  yhat_lower

 52%|████████████████████████████████████████▎                                    | 22/42 [5:48:31<5:16:50, 950.50s/it]


getting best preds for  c0
2015-10-01 00:00:00
too many zero values, return default best params


ValueError: Dataframe has less than 2 non-NaN rows.